<br>
<br>

<p style='color:red;'><b>본 실습 중 다뤄지는 코드의 경우 네이버 뉴스의 정책 및 소스코드 변경에 따라 일부가 정상적으로 작동하지 않을 수 있습니다.</b></p>

In [1]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 KB 19.4 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup # HTTP Response -> HTML 

import pandas as pd
from datetime import datetime
import time 
import re

<br>
<br>

## 1. 뉴스 검색 결과에서 네이버뉴스 추려내기

In [5]:
query = '데이터분석'
url = "https://search.naver.com/search.naver?where=news&query=" + query

![image.png](attachment:image.png)

In [4]:
# web = urlopen(url)
# print(web)

In [6]:
web = requests.get(url).content # request methods : get / post / put / delete ...
#print(web)

In [7]:
source = BeautifulSoup(web, 'html.parser')
# print(source) # source를 그대로 출력할 경우, 담고 있는 텍스트가 무척 많아서 버벅이게 될 수 있습니다.

<!DOCTYPE html>
 <html lang="ko"> <head> <meta charset="utf-8"/> <meta content="always" name="referrer"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <meta content="width=device-width,initial-scale=1.0,maximum-scale=2.0" name="viewport"/> <meta content="데이터분석 : 네이버 뉴스검색" property="og:title"> <meta content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png" property="og:image"/> <meta content="'데이터분석'의 네이버 뉴스검색 결과입니다." property="og:description"/> <meta content="'데이터분석'의 네이버 뉴스검색 결과입니다." lang="ko" name="description"/> <title>데이터분석 : 네이버 뉴스검색</title> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico" rel="shortcut icon"/> <link href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" rel="search" title="Naver" type="application/opensearchdescription+xml"><link href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_230216.css" rel="stylesheet" type="text/css"/> <link href="https://ssl.pstatic.net

![image.png](attachment:image.png)

In [7]:
# 네이버에서 "데이터분석"을 검색한 후 뉴스 탭의 1번째 페이지(위 url 변수의 URL에 해당)에 나타나 있는 뉴스들의 "제목"을 크롤링해주세요. 
# 텍스트만 뽑아내어 하나의 리스트로 모아 저장해주세요. (ex. 위 스크린샷에서 [인천테크노파크, 중소기업 빅데이터 지원사업 '우수' 등급])

#news_subjects = source.find_all("a", {"class" : "news_tit"}) # ResultSet (리스트와 유사한 형태)

#subject_list = []

#for subject in news_subjects:
#    subject_list.append(subject.get_text())

#print(subject_list)
#print(news_subjects)

In [8]:
#urls = news_subjects
#first_article = urls[0]
#first_article.attrs["href"] # tag's attributes (== attrs)

In [9]:
for urls in source.find_all('a', {'class' : 'info'}):
    if len(urls.attrs['class'])==1:
        print(urls.attrs['href'])

https://n.news.naver.com/mnews/article/057/0001731022?sid=101
https://n.news.naver.com/mnews/article/001/0013835173?sid=105
https://n.news.naver.com/mnews/article/009/0005106262?sid=105
https://n.news.naver.com/mnews/article/092/0002286560?sid=105
https://n.news.naver.com/mnews/article/003/0011755971?sid=104
https://n.news.naver.com/mnews/article/015/0004824077?sid=105
https://n.news.naver.com/mnews/article/079/0003750751?sid=102


In [10]:
# sample_url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=014&aid=0004286058'
# sample_url.?('https://news.naver.com/')

sample_url = 'https://n.news.naver.com/mnews/article/003/0011379817?sid=102' # 네이버뉴스 URL이 변경되었습니다.
sample_url.startswith('https://n.news.naver.com/')

True

In [11]:
urls_list = []

for urls in source.find_all('a', {'class' : "info"}):
    if urls.attrs["href"].startswith("https://n.news.naver.com"): # starts with ~~~ # 네이버뉴스 URL이 변경되었습니다.
        urls_list.append(urls.attrs["href"])

urls_list

['https://n.news.naver.com/mnews/article/057/0001731022?sid=101',
 'https://n.news.naver.com/mnews/article/001/0013835173?sid=105',
 'https://n.news.naver.com/mnews/article/009/0005106262?sid=105',
 'https://n.news.naver.com/mnews/article/092/0002286560?sid=105',
 'https://n.news.naver.com/mnews/article/003/0011755971?sid=104',
 'https://n.news.naver.com/mnews/article/015/0004824077?sid=105',
 'https://n.news.naver.com/mnews/article/079/0003750751?sid=102']

<br>
<br>

## 2. 단일 뉴스 페이지 분석하기

In [12]:
# # 아래 코드를 실행하면 네이버에서 Bot으로 인식하여 접속을 차단하므로 'ConnectionError'가 발생합니다

#web_news = requests.get(urls_list[0]).content
#source_news = BeautifulSoup(web_news, 'html.parser')

In [13]:
# 따라서 아래와 같이 코드를 수정하여 크롬브라우저에서 보내는 요청으로 인식하도록 HTTP Request에 Header 정보를 추가해주면 됩니다.

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

web_news = requests.get(urls_list[0], headers=headers).content 
source_news = BeautifulSoup(web_news, 'html.parser')

<br>
<br>

#### 1) 기사 제목 & 발행 날짜

![image.png](attachment:image.png)

In [14]:
title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
print(title)

3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국]


In [15]:
date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()
date = date.replace(" ","")
date1 = date[:11] # 날짜까지 파싱
date2 = date[13:] # 시간부분 파싱
date3 = (lambda x : 'am' if x == '오전' else 'pm')(date[11:13])
date4 = date1 + date2 + date3
date = pd.Timestamp(date4)
print(date)

2023-03-25 09:00:00


In [16]:
# kor_to_eng = lambda x : 'am' if x == '오전' else 'pm'

In [17]:
# def kor_to_eng(x):
#     if x == '오전':
#         return 'am'
#     else:
#         return 'pm'

In [18]:
# date3 = kor_to_eng(date[11:13])

<br>
<br>

#### 2) 기사 본문

![image.png](attachment:image.png)

In [19]:
# ArticleBodyContents -> CamelCase (Python에서는 보통 클래스명에만 사용)
# article_body_contents -> snake_case

article = source_news.find('div', {'id' : 'dic_area'}).get_text()
article
# print(article)

'\n\n\n\n\n지난 2월 1일부터 서울의 택시 요금이 인상됐습니다. 중형택시를 기준으로 3,800원이었던 기본요금이 4,800원으로 26% 넘게 올랐고요. 기본요금 거리도 2km애서 1.6km로 줄었습니다. 거리요금 기준도 132m당 100원에서 131m당 100원, 시간요금 기준도 31초당 100원에서 20초당 100원으로 인상됐죠.2019년 이후 4년만의 인상인데, 이를 두고 택시 업계의 반응은 엇갈렸습니다. 운송 원가에 비해 지나치게 낮은 택시 요금을 정상화한다며 반기는 사람도 있었고, 요금 인상으로 손님이 줄 것이란 우려도 나왔습니다. 실제로 요금 인상이 이뤄진 직후 손님이 줄었다는 보도도 줄을 이었죠.실제론 어땠을까요? <데이터로 본 대한민국>에서 KDX한국데이터거래소의 앱 이용자 분석 서비스인 어데고(adego)를 이용해서 분석해봤는데요. 대형 택시호출 앱의 주간 이용자(WAU) 데이터를 활용했습니다.추웠던 2월은 잘 버텼다, 하지만 꽃피는 3월은... \n\n\n\n1월 첫주부터 2월 마지막 주까지 주차별로 택시호출 앱 이용자 수(WAU)를 표시한 그래프입니다. 언뜻 보면 택시 요금 인상이 이뤄진 이후 이용자 수가 대폭 감소한 것처럼 보이는데요. 실제로 1월 평균 WAU는 395만 명이고, 2월의 평균 WAU는 370만으로 약 6.2%가 감소하긴 했습니다.하지만 지난해 수치와 비교해보면 어떨까요? \n\n\n\n엎치락 뒤치락 꽤 비슷한 모습을 보이는데요. 주차별로 따져보면 WAU는 2월 2주와 3주가 3% 미만의 감소율을 보였고, 2월 1주와 4주는 1%도 채 감소하지 않았습니다.그렇다면, 2월에는 택시 요금 인상이 이용자수에 큰 타격을 주진 않았다는 건데, 3월에는 어땠을까요?\n\n\n\n3월 첫 주는 6.1% 감소했고, 둘째 주는 7.4% 감소했습니다. 그리고 3월 셋째주에는 무려 33% 넘게 폭락했죠. 우려했던 택시 이용객 수 감소가 3월 들어서 본격적으로 나타나기 시작한 겁니다.값 비싼 택시 대신 자전거, 공유차 수요 급증택시 수요가

In [20]:
article = article.replace("\n", "") # 교체하다
article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
article = article.replace("동영상 뉴스       ", "")
article = article.replace("동영상 뉴스", "")
article = article.strip()
article

'지난 2월 1일부터 서울의 택시 요금이 인상됐습니다. 중형택시를 기준으로 3,800원이었던 기본요금이 4,800원으로 26% 넘게 올랐고요. 기본요금 거리도 2km애서 1.6km로 줄었습니다. 거리요금 기준도 132m당 100원에서 131m당 100원, 시간요금 기준도 31초당 100원에서 20초당 100원으로 인상됐죠.2019년 이후 4년만의 인상인데, 이를 두고 택시 업계의 반응은 엇갈렸습니다. 운송 원가에 비해 지나치게 낮은 택시 요금을 정상화한다며 반기는 사람도 있었고, 요금 인상으로 손님이 줄 것이란 우려도 나왔습니다. 실제로 요금 인상이 이뤄진 직후 손님이 줄었다는 보도도 줄을 이었죠.실제론 어땠을까요? <데이터로 본 대한민국>에서 KDX한국데이터거래소의 앱 이용자 분석 서비스인 어데고(adego)를 이용해서 분석해봤는데요. 대형 택시호출 앱의 주간 이용자(WAU) 데이터를 활용했습니다.추웠던 2월은 잘 버텼다, 하지만 꽃피는 3월은... 1월 첫주부터 2월 마지막 주까지 주차별로 택시호출 앱 이용자 수(WAU)를 표시한 그래프입니다. 언뜻 보면 택시 요금 인상이 이뤄진 이후 이용자 수가 대폭 감소한 것처럼 보이는데요. 실제로 1월 평균 WAU는 395만 명이고, 2월의 평균 WAU는 370만으로 약 6.2%가 감소하긴 했습니다.하지만 지난해 수치와 비교해보면 어떨까요? 엎치락 뒤치락 꽤 비슷한 모습을 보이는데요. 주차별로 따져보면 WAU는 2월 2주와 3주가 3% 미만의 감소율을 보였고, 2월 1주와 4주는 1%도 채 감소하지 않았습니다.그렇다면, 2월에는 택시 요금 인상이 이용자수에 큰 타격을 주진 않았다는 건데, 3월에는 어땠을까요?3월 첫 주는 6.1% 감소했고, 둘째 주는 7.4% 감소했습니다. 그리고 3월 셋째주에는 무려 33% 넘게 폭락했죠. 우려했던 택시 이용객 수 감소가 3월 들어서 본격적으로 나타나기 시작한 겁니다.값 비싼 택시 대신 자전거, 공유차 수요 급증택시 수요가 줄었다면 반대로 이용객이 늘어난 것도 있을텐데요. 바로 공유

In [21]:
article

'지난 2월 1일부터 서울의 택시 요금이 인상됐습니다. 중형택시를 기준으로 3,800원이었던 기본요금이 4,800원으로 26% 넘게 올랐고요. 기본요금 거리도 2km애서 1.6km로 줄었습니다. 거리요금 기준도 132m당 100원에서 131m당 100원, 시간요금 기준도 31초당 100원에서 20초당 100원으로 인상됐죠.2019년 이후 4년만의 인상인데, 이를 두고 택시 업계의 반응은 엇갈렸습니다. 운송 원가에 비해 지나치게 낮은 택시 요금을 정상화한다며 반기는 사람도 있었고, 요금 인상으로 손님이 줄 것이란 우려도 나왔습니다. 실제로 요금 인상이 이뤄진 직후 손님이 줄었다는 보도도 줄을 이었죠.실제론 어땠을까요? <데이터로 본 대한민국>에서 KDX한국데이터거래소의 앱 이용자 분석 서비스인 어데고(adego)를 이용해서 분석해봤는데요. 대형 택시호출 앱의 주간 이용자(WAU) 데이터를 활용했습니다.추웠던 2월은 잘 버텼다, 하지만 꽃피는 3월은... 1월 첫주부터 2월 마지막 주까지 주차별로 택시호출 앱 이용자 수(WAU)를 표시한 그래프입니다. 언뜻 보면 택시 요금 인상이 이뤄진 이후 이용자 수가 대폭 감소한 것처럼 보이는데요. 실제로 1월 평균 WAU는 395만 명이고, 2월의 평균 WAU는 370만으로 약 6.2%가 감소하긴 했습니다.하지만 지난해 수치와 비교해보면 어떨까요? 엎치락 뒤치락 꽤 비슷한 모습을 보이는데요. 주차별로 따져보면 WAU는 2월 2주와 3주가 3% 미만의 감소율을 보였고, 2월 1주와 4주는 1%도 채 감소하지 않았습니다.그렇다면, 2월에는 택시 요금 인상이 이용자수에 큰 타격을 주진 않았다는 건데, 3월에는 어땠을까요?3월 첫 주는 6.1% 감소했고, 둘째 주는 7.4% 감소했습니다. 그리고 3월 셋째주에는 무려 33% 넘게 폭락했죠. 우려했던 택시 이용객 수 감소가 3월 들어서 본격적으로 나타나기 시작한 겁니다.값 비싼 택시 대신 자전거, 공유차 수요 급증택시 수요가 줄었다면 반대로 이용객이 늘어난 것도 있을텐데요. 바로 공유

#### (Appendix) 불필요한 기호와 기자 이메일 주소 등 삭제하기 

<br>

- 네이버뉴스 하단의 **기자 정보**가 **byline** div 태그로 분리되면서 아래 정규표현식을 활용한 후처리는 불필요하게 되었습니다.
- 이에 필요 시 샘플 텍스트를 기준으로 설명을 진행합니다.

In [22]:
# news_contents = '[인천=뉴시스] 이루비 기자 =  인천테크노파크 로고. (이미지=인천테크노파크 제공)[인천=뉴시스] 이루비 기자 = 인천테크노파크는 \'중소기업 빅데이터 분석·활용 지원사업\' 전국 평가에서 \'우수\' 등급을 받았다고 2일 밝혔다. 이 지원사업은 중소기업과 빅데이터 분석 솔루션을 보유한 데이터 전문기업을 연결해 기업의 빅데이터 분석·활용을 돕는 것으로, 인천TP는 지난 2020년부터 한국지능정보사회진흥원(NIA)의 공모에 2년 연속 선정돼 사업을 추진해 왔다.인천TP는 지난해 사업비 5억원을 들여 전문성 부족과 비용 문제 등으로 빅데이터 활용에 어려움을 겪고 있는 인천지역 28개 중소기업에 데이터 전문기업의 맞춤형 컨설팅을 지원, 기업 성장을 이끌었다.스마트 가로등 개발업체 ㈜에코란트는 인천TP의 빅데이터 분석·활용 지원을 바탕으로 매출이 두배 넘게 늘었으며, 유아용품 전문 제조업 ㈜제이엠아이엔티엘은 신제품 출시를 앞당겼다. 이사 플랫폼 서비스 제공기업 ㈜모두플랫폼과 공기압 기기 국내 최대 제조업체 ㈜TPC메카트로닉스, 유압 실린더 개발 기업 우영유압㈜은 생산관리 효율성 향상으로 시장경쟁력을 높였다. 운동 어플 서비스 개발업체 ㈜버닛과 AI(인공지능) 기반 폐기물 분류 시스템 개발 기업 ㈜에이트테크는 작업효율 높이면서 투자유치에 성공, 사세 확장을 꾀하고 있다.인천TP 관계자는 "이 지원사업을 통해 기업들이 새로운 비즈니스 기회를 창출하면서 경쟁력을 키워나가고 있다"며 "인천지역 기업들의 데이터 활용 활성화와 비즈니스 모델 전환 등을 지속해서 지원할 계획"이라고 말했다．eswoo@newsis.com▶ 네이버에서 뉴시스 채널 구독하기▶ 뉴시스 빅데이터 MSI 주가시세표 바로가기 <저작권자ⓒ 공감언론 뉴시스통신사. 무단전재-재배포 금지.>'
# news_contents

In [23]:
# import re

# re.match('^[a-zA-Z]+', 'airplane')
# re.match('^[a-zA-Z]+', '?')

In [24]:
# pattern = re.compile(r'[\s\Wa-zA-Z0-9]*@') 
# # [white-space, anti-alphanumeric, lower/upper, numeric] 중에서 0 or more + "@"
# # 정규표현식 추가자료 1) 정규표현식 개념 소개 & 기본 문법 정리 @ https://j.mp/3bJQJHg & https://j.mp/3bLXSqB
# # 정규표현식 추가자료 2) 보다 상세한 설명 & 예제 : http://j.mp/2PzgFO8 & https://hamait.tistory.com/342
# # 정규표현식 추가자료 3) AutoRegex (Convert english to RegEx with natural language processing) : https://www.autoregex.xyz

# email_address = pattern.search(news_contents)

# print(email_address)
# print(email_address.start())
# print(news_contents[:email_address.start()])

In [25]:
# # 1) "\'" 기호 삭제하기 
# pattern = re.compile(r'\'') 
# news_contents = pattern.sub('', news_contents) # substitute 

# # 2) 기자 이메일 주소부터 그 이후 전체 삭제하기
# pattern = re.compile(r'[\s\Wa-zA-Z0-9]*@')  
# email_address = pattern.search(news_contents)
# news_contents = news_contents[:email_address.start()]

# print(news_contents)

<br>
<br>

#### 3) 기사 발행 언론사

![image-2.png](attachment:image-2.png)

In [26]:
press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
print(press_company)

MBN


<br>
<br>

## 3-1. 여러 뉴스 데이터 모으기

In [27]:
query = '데이터분석'
url = "https://search.naver.com/search.naver?where=news&query=" + query
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

urls_list = []

for urls in source.find_all('a', {'class' : "info"}):
    if urls["href"].startswith("https://n.news.naver.com"):
        urls_list.append(urls["href"])

urls_list

['https://n.news.naver.com/mnews/article/057/0001731022?sid=101',
 'https://n.news.naver.com/mnews/article/001/0013835173?sid=105',
 'https://n.news.naver.com/mnews/article/009/0005106262?sid=105',
 'https://n.news.naver.com/mnews/article/092/0002286560?sid=105',
 'https://n.news.naver.com/mnews/article/003/0011755971?sid=104',
 'https://n.news.naver.com/mnews/article/015/0004824077?sid=105',
 'https://n.news.naver.com/mnews/article/079/0003750751?sid=102']

In [28]:
# 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)

titles = []
dates = []
articles = []
article_urls = []
press_companies = []

In [29]:
query = '데이터분석'
url = "https://search.naver.com/search.naver?where=news&query=" + query
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

# 1) 네이버 뉴스만 추려내기
urls_list = []
for urls in source.find_all('a', {'class' : "info"}):
    if urls["href"].startswith("https://n.news.naver.com"):
        urls_list.append(urls["href"])

for url in urls_list:
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    web_news = requests.get(url, headers=headers).content
    source_news = BeautifulSoup(web_news, 'html.parser')

    # 2) 기사 제목 
    title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
    titles.append(title)
    print('Processing article : {}'.format(title))
    
    # 3) 기사 날짜
    date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()
    dates.append(date)

    # 4) 기사 본문
    article = source_news.find('div', {'id' : 'dic_area'}).get_text()
    article = article.replace("\n", "")
    article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
    article = article.replace("동영상 뉴스       ", "")
    article = article.replace("동영상 뉴스", "")
    article = article.strip()
    articles.append(article)
    
    # 5) 기사 URL 
    article_urls.append(url)
    
    # 6) 기사 발행 언론사
    press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
    press_companies.append(press_company)

Processing article : 3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국]
Processing article : [게시판] 네이버, 쇼핑데이터로 건강기능식품 추천 기획전
Processing article : 올해 뉴스 분석하는 챗GPT가 있다고?…챗GPT 플러그인 출시
Processing article : KTNET-산업연구원, 데이터 기반 사업 협력
Processing article : 구글 인공지능 바드, 챗GPT보다 훨씬 신중-NYT분석
Processing article : 인바디, 전 세계 체성분 분석 빅데이터 보고서 발간
Processing article : 광주, '치매 데이터 활용 기반구축' 본격화


In [30]:
# 결과 체크 

print(press_companies)

['MBN', '연합뉴스', '매일경제', 'ZDNet Korea', '뉴시스', '한국경제', '노컷뉴스']


In [31]:
print(datetime.now())
print(datetime.now().strftime('%y%m%d_%H%M')) # string format time

2023-03-25 09:24:24.775032
230325_0924


In [32]:
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.

article_df = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

article_df.to_csv('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
article_df.head()

,Title,Date,Article,URL,PressCompany
0,3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국],2023.03.25. 오전 9:00,"지난 2월 1일부터 서울의 택시 요금이 인상됐습니다. 중형택시를 기준으로 3,800...",https://n.news.naver.com/mnews/article/057/000...,MBN
1,"[게시판] 네이버, 쇼핑데이터로 건강기능식품 추천 기획전",2023.03.24. 오전 10:50,네이버 헬시차트 기획전[네이버 제공. 재판매 및 DB 금지]▲ 네이버는 지난해 1월...,https://n.news.naver.com/mnews/article/001/001...,연합뉴스
2,올해 뉴스 분석하는 챗GPT가 있다고?…챗GPT 플러그인 출시,2023.03.24. 오전 10:41,챗GPT 브라우징 가능한 서비스 내놔최신 영화 정보 및 뉴스 등 자동 분석11개 기...,https://n.news.naver.com/mnews/article/009/000...,매일경제
3,"KTNET-산업연구원, 데이터 기반 사업 협력",2023.03.24. 오후 3:01,무역·통상정책 공동연구 추진 및 수출 데이터 활용 MOU한국무역정보통신(KTNET·...,https://n.news.naver.com/mnews/article/092/000...,ZDNet Korea
4,"구글 인공지능 바드, 챗GPT보다 훨씬 신중-NYT분석",2023.03.22. 오전 9:54,기사내용 요약스스로 한계 인정하며 인물·의학 답변은 회피[서울=뉴시스]구글이 인공지...,https://n.news.naver.com/mnews/article/003/001...,뉴시스


<br>
<br>

## 3-2. 여러 뉴스 데이터 모으기 (특정 뉴스 웹페이지 크롤링 중 에러 발생 시 회피)

In [33]:
query = '데이터분석'
url = "https://search.naver.com/search.naver?where=news&query=" + query
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

# 1) 네이버 뉴스만 추려내기
urls_list = []
for urls in source.find_all('a', {'class' : "info"}):
    if urls["href"].startswith("https://n.news.naver.com"):
        urls_list.append(urls["href"])

for url in urls_list:
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web_news = requests.get(url, headers=headers).content
        source_news = BeautifulSoup(web_news, 'html.parser')

        # 2) 기사 제목 
        title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
        print('Processing article : {}'.format(title))

        # 3) 기사 날짜
        date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

        # 4) 기사 본문
        article = source_news.find('div', {'id' : 'dic_area'}).get_text()
        article = article.replace("\n", "")
        article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
        article = article.replace("동영상 뉴스       ", "")
        article = article.replace("동영상 뉴스", "")
        article = article.strip()
        
        # 5) 기사 발행 언론사
        press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
        
        # 위 2~5를 통해 성공적으로 제목/날짜/본문/언론사 정보가 모두 추출되었을 때에만 리스트에 추가해 길이를 동일하게 유지해줍니다.
        titles.append(title)
        dates.append(date)
        articles.append(article)
        press_companies.append(press_company)
        article_urls.append(url) # 6) 기사 URL 
    
    except:
        print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
        

# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모아둡니다.
article_df = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

# 파일명을 result_연도월일_시분.csv 로 지정합니다.
article_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
article_df.head()

Processing article : 3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국]
Processing article : [게시판] 네이버, 쇼핑데이터로 건강기능식품 추천 기획전
Processing article : 올해 뉴스 분석하는 챗GPT가 있다고?…챗GPT 플러그인 출시
Processing article : KTNET-산업연구원, 데이터 기반 사업 협력
Processing article : 구글 인공지능 바드, 챗GPT보다 훨씬 신중-NYT분석
Processing article : 인바디, 전 세계 체성분 분석 빅데이터 보고서 발간
Processing article : 광주, '치매 데이터 활용 기반구축' 본격화


,Title,Date,Article,URL,PressCompany
0,3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국],2023.03.25. 오전 9:00,"지난 2월 1일부터 서울의 택시 요금이 인상됐습니다. 중형택시를 기준으로 3,800...",https://n.news.naver.com/mnews/article/057/000...,MBN
1,"[게시판] 네이버, 쇼핑데이터로 건강기능식품 추천 기획전",2023.03.24. 오전 10:50,네이버 헬시차트 기획전[네이버 제공. 재판매 및 DB 금지]▲ 네이버는 지난해 1월...,https://n.news.naver.com/mnews/article/001/001...,연합뉴스
2,올해 뉴스 분석하는 챗GPT가 있다고?…챗GPT 플러그인 출시,2023.03.24. 오전 10:41,챗GPT 브라우징 가능한 서비스 내놔최신 영화 정보 및 뉴스 등 자동 분석11개 기...,https://n.news.naver.com/mnews/article/009/000...,매일경제
3,"KTNET-산업연구원, 데이터 기반 사업 협력",2023.03.24. 오후 3:01,무역·통상정책 공동연구 추진 및 수출 데이터 활용 MOU한국무역정보통신(KTNET·...,https://n.news.naver.com/mnews/article/092/000...,ZDNet Korea
4,"구글 인공지능 바드, 챗GPT보다 훨씬 신중-NYT분석",2023.03.22. 오전 9:54,기사내용 요약스스로 한계 인정하며 인물·의학 답변은 회피[서울=뉴시스]구글이 인공지...,https://n.news.naver.com/mnews/article/003/001...,뉴시스


<br>
<br>

## 4. 여러 페이지에 걸쳐 크롤링하기

![image.png](attachment:image.png)

In [34]:
query = '데이터분석'
url = "https://search.naver.com/search.naver?where=news&query=" + query + '&start' + str(11) # 2 페이지
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

urls_list = []
for urls in source.find_all('a', {'class' : "info"}):
    if urls["href"].startswith("https://n.news.naver.com"):
        urls_list.append(urls["href"])

for url in urls_list:
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web_news = requests.get(url, headers=headers).content
        source_news = BeautifulSoup(web_news, 'html.parser')

        title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
        titles.append(title)
        print(title)
    except:
        print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))

3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국]
[게시판] 네이버, 쇼핑데이터로 건강기능식품 추천 기획전
올해 뉴스 분석하는 챗GPT가 있다고?…챗GPT 플러그인 출시
KTNET-산업연구원, 데이터 기반 사업 협력
구글 인공지능 바드, 챗GPT보다 훨씬 신중-NYT분석
인바디, 전 세계 체성분 분석 빅데이터 보고서 발간
광주, '치매 데이터 활용 기반구축' 본격화


In [35]:
'''
query = '데이터분석'
url = "https://search.naver.com/search.naver?where=news&query=" + query + ? + ?(21) # 3 페이지
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

urls_list = []
for urls in source.find_all('a', {'class' : "info"}):
    if urls["href"].startswith("https://n.news.naver.com"):
        urls_list.append(urls["href"])

for url in urls_list:
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web_news = requests.get(url, headers=headers).content
        source_news = BeautifulSoup(web_news, 'html.parser')

        title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
        titles.append(title)
        print(title)
    except:
        print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
        '''

'\nquery = \'데이터분석\'\nurl = "https://search.naver.com/search.naver?where=news&query=" + query + ? + ?(21) # 3 페이지\nweb = requests.get(url).content\nsource = BeautifulSoup(web, \'html.parser\')\n\nurls_list = []\nfor urls in source.find_all(\'a\', {\'class\' : "info"}):\n    if urls["href"].startswith("https://n.news.naver.com"):\n        urls_list.append(urls["href"])\n\nfor url in urls_list:\n    try:\n        headers = {\'User-Agent\':\'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36\'}\n        web_news = requests.get(url, headers=headers).content\n        source_news = BeautifulSoup(web_news, \'html.parser\')\n\n        title = source_news.find(\'h2\', {\'class\' : \'media_end_head_headline\'}).get_text()\n        titles.append(title)\n        print(title)\n    except:\n        print(\'*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}\'.format(url))\n        '

In [36]:
max_page = 5
start_points = []

for point in range(1, max_page*10, 10): # range(start, stop, step)
    start_points.append(str(point)) 

start_points

['1', '11', '21', '31', '41']

In [37]:
query = '데이터분석'
max_page = 5 # 크롤링을 원하는 최대 페이지 수 지정

In [38]:
# 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)

titles = []
dates = []
articles = []
article_urls = []
press_companies = []

In [39]:
# current_call = 1
# last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

# while current_call <= last_call:
#     print(current_call) # 1, 11, 21, 31, 41 
#     current_call += 10

# # current_call 1 <= 41
# # current_call 11 <= 41
# # current_call 21 <= 41
# # current_call 31 <= 41
# # current_call 41 <= 41
# # current_call 51 <= 41 <- False & 종료

In [47]:
current_call = 1
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call: # 조건문이 참인 "동안" 실행
    
    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query + "&start=" + str(current_call)
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class' : "info"}):
        if urls["href"].startswith("https://n.news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
            print('Processing article : {}'.format(title))

            date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

            article = source_news.find('div', {'id' : 'dic_area'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()

            press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)
        except:
            print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
        time.sleep(1)
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
article_df = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

article_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
article_df.head()


1번째 기사글부터 크롤링을 시작합니다.
Processing article : 3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국]
Processing article : [게시판] 네이버, 쇼핑데이터로 건강기능식품 추천 기획전
Processing article : 올해 뉴스 분석하는 챗GPT가 있다고?…챗GPT 플러그인 출시
Processing article : KTNET-산업연구원, 데이터 기반 사업 협력
Processing article : 구글 인공지능 바드, 챗GPT보다 훨씬 신중-NYT분석
Processing article : 인바디, 전 세계 체성분 분석 빅데이터 보고서 발간
Processing article : 광주, '치매 데이터 활용 기반구축' 본격화

11번째 기사글부터 크롤링을 시작합니다.
Processing article : "데이터시장 규모 2027년 50조…경남 통합데이터 구축 필요"
Processing article : 한전, 전력 데이터 활용 활성화 위한 설명회 개최
Processing article : 메디에이지-스위스리 한국지점, 의료 빅데이터 기반 공동연구 협약 체결
Processing article : [단독] 사라진 1등 복권…위치 알 유통 데이터도 열었다 (끝까지판다 풀영상)
Processing article : 대구사이버대학교, 학습자 분석 시스템 도입…학업 유지율 높여
Processing article : 네이버, 쇼핑데이터로 건강기능식품 추천… ‘헬시차트’ 기획전
Processing article : 韓 첫 민간발사체 '한빛-TLV' 무사 발사…성공 여부 분석 중
Processing article : 북한 인권·분석 강화, 남북 교류·대화 축소···통일부 대대적 조직개편
Processing article : 워크데이 ‘방대한 데이터·인공지능 기술로 최적의 인사이트 제공’

21번째 기사글부터 크롤링을 시작합니다.
Processing article : [사실은] "일본 수십 차례 사과" 전

,Title,Date,Article,URL,PressCompany
0,"AI가 핫하다는데 어디에 투자를, 110조로 커지는 이 시장 [더테크웨이브]",2023.03.25. 오후 6:05,클래리베이트 ‘글로벌 AI반도체 혁신 보고서’ AI 반도체 [사진=연합뉴스]최근 챗...,https://n.news.naver.com/mnews/article/009/000...,매일경제
1,[건강당당] 숫자로 알아보는 당뇨,2023.03.25. 오후 5:01,"[그래픽=최소연 디자이너]고려대 의대 윤석준 교수팀은 국민건강보험, 중앙암등록본부,...",https://n.news.naver.com/mnews/article/296/000...,코메디닷컴
2,"VR로 재탄생한 1978년 게임, 집처럼 편안한 혼합현실..""VR은 살아있다""",2023.03.25. 오후 4:56,[GDC23]게임개발자 콘퍼런스서 VR 기업 존재감고전게임 '시조새' 아타리 VR게...,https://n.news.naver.com/mnews/article/018/000...,이데일리
3,"‘미국 퇴출 위기’ 틱톡, 정작 중국인은 ‘이것’ 때문에 안 쓴다 [핫이슈]",2023.03.25. 오후 4:28,"[서울신문 나우뉴스]왼쪽 더우인, 오른쪽 틱톡 미국 정치권에서 중국산 동영상 플랫폼...",https://n.news.naver.com/mnews/article/081/000...,서울신문
4,"美 퇴출위기 맞은 틱톡… ""바이든 최측근 고용했다""",2023.03.25. 오후 3:36,미국 시장에서 퇴출 압박을 받고 있는 틱톡이 조 바이든 대통령의 측근들을 잇따라 고...,https://n.news.naver.com/mnews/article/417/000...,머니S


<br>
<br>

## (Appendix) 날짜 지정하여 크롤링하기

In [41]:
query = '데이터분석'

start_date = '2022.01.01'
end_date = '2022.01.30'

max_page = 5 # 크롤링을 원하는 최대 페이지 수 지정

In [42]:
# 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
titles = []
dates = []
articles = []
article_urls = []
press_companies = []


# 주어진 일자를 쿼리에 맞는 형태로 변경해줍니다.
start_date = start_date.replace(".", "")
end_date = end_date.replace(".", "")

# 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
current_call = 1
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call:
    
    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query \
          + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
          + "to" + end_date \
          + "%2Ca%3A&start=" + str(current_call)

    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class' : "info"}):
        if urls["href"].startswith("https://n.news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
            print('Processing article : {}'.format(title))

            date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

            article = source_news.find('div', {'id' : 'dic_area'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()

            press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)
        except:
            print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    time.sleep(5)
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
article_df = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

article_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
article_df.head()


1번째 기사글부터 크롤링을 시작합니다.
Processing article : KT, '빅데이터 상권분석'으로 소상공인 돕는다
Processing article : [싱글 파이낸스]② 마이데이터로 계획 세우고 IRP로 노후 대비
Processing article : 국가 AI데이터센터, AI 핵심거점으로 자리매김
*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : https://n.news.naver.com/mnews/article/112/0003526525?sid=106
Processing article : 진짜 고기 뺨치는 가짜고기 전쟁… 버섯으로 식감 살리고, AI로 분자분석
Processing article : LS그룹, AI·빅데이터·스마트에너지 접목…디지털 기업 전환 촉진
Processing article : 부산관광공사, 관광 공공데이터 225만건 개방

11번째 기사글부터 크롤링을 시작합니다.
Processing article : KT, 빅데이터 상권분석 플랫폼 '잘나가게' 활용법 제공
Processing article : "클라우드는 수단일 뿐, 데이터 통찰력 확보가 목표여야"
Processing article : "한국 데이터 가상화 시장 새해 급성장할 것"
*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : https://n.news.naver.com/mnews/article/112/0003526525?sid=106
Processing article : 식․의약 데이터 융합․분석 플랫폼 구축 4월부터 본격화
Processing article : 부산진구, 데이터기반 통합플랫폼 구축으로 스마트시티 조성
Processing article : [테크쑤다]"마이데이터 활용하면, 주린이도 고수익률"…핀크 권영탁 대표
Processing article : 진짜 고기 뺨치는 가짜고기 전쟁… 버섯으로 식감 살리고, AI로 분자분석

21번째 기사글부터 크롤링을 시작합니다.
Processing article : W컨셉, 일요일 '벼

,Title,Date,Article,URL,PressCompany
0,"KT, '빅데이터 상권분석'으로 소상공인 돕는다",2022.01.28. 오후 2:56,기사내용 요약KT 빅데이터 상권분석 활용법 알려주는 '잘나가게 이야기'누구나 데이터...,https://n.news.naver.com/mnews/article/003/001...,뉴시스
1,[싱글 파이낸스]② 마이데이터로 계획 세우고 IRP로 노후 대비,2022.01.30. 오전 6:01,‘1코노미’ ‘혼밥’ ‘혼술’ 등 따지고 보면 1인가구란 화두는 2010년대 중반부...,https://n.news.naver.com/mnews/article/366/000...,조선비즈
2,"국가 AI데이터센터, AI 핵심거점으로 자리매김",2022.01.30. 오후 4:40,지난해 125개 연구과제 지원…만족도 향상·매출 등 성과국가 AI데이터센터 조감도....,https://n.news.naver.com/mnews/article/421/000...,뉴스1
3,"진짜 고기 뺨치는 가짜고기 전쟁… 버섯으로 식감 살리고, AI로 분자분석",2022.01.30. 오후 12:01,채식주의자뿐 아니라 MZ 세대까지 식물성 식품에 열광하면서 대체육 기술 개발 경쟁이...,https://n.news.naver.com/mnews/article/023/000...,조선일보
4,"LS그룹, AI·빅데이터·스마트에너지 접목…디지털 기업 전환 촉진",2022.01.30. 오후 2:14,[서울=뉴시스] 옥승욱 기자 = 구자은 LS그룹 회장은 올해 1월 LS 회장에 취임...,https://n.news.naver.com/mnews/article/003/001...,뉴시스


<br>
<br>

## (Appendix) 기사 정렬 순서 지정하여 크롤링하기

In [43]:
query = '데이터분석'

sort_type = 1 # 관련도순 = 0  최신순 = 1  오래된순 = 2

max_page = 5 # 크롤링을 원하는 최대 페이지 수 지정

In [0]:
# 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
titles = []
dates = []
articles = []
article_urls = []
press_companies = []


# 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
current_call = 1
last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 

while current_call <= last_call:
    
    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
    
    url = "https://search.naver.com/search.naver?where=news&query=" + query \
          + "&sort=" + str(sort_type) \
          + "&start=" + str(current_call)

    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class' : "info"}):
        if urls["href"].startswith("https://n.news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        try:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
            print('Processing article : {}'.format(title))

            date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

            article = source_news.find('div', {'id' : 'dic_area'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()

            press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
            
            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)
            time.sleep(5)
        except:
            print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
            
    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    current_call += 10

    
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
article_df = pd.DataFrame({'Title':titles, 
                           'Date':dates, 
                           'Article':articles, 
                           'URL':article_urls, 
                           'PressCompany':press_companies})

article_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
article_df.head()


1번째 기사글부터 크롤링을 시작합니다.
Processing article : AI가 핫하다는데 어디에 투자를, 110조로 커지는 이 시장 [더테크웨이브]
Processing article : [건강당당] 숫자로 알아보는 당뇨
Processing article : VR로 재탄생한 1978년 게임, 집처럼 편안한 혼합현실.."VR은 살아있다"
Processing article : ‘미국 퇴출 위기’ 틱톡, 정작 중국인은 ‘이것’ 때문에 안 쓴다 [핫이슈]
*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : https://n.news.naver.com/mnews/article/081/0003348758?sid=104

11번째 기사글부터 크롤링을 시작합니다.
Processing article : 美 퇴출위기 맞은 틱톡… "바이든 최측근 고용했다"
*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : https://n.news.naver.com/mnews/article/311/0001573188?sid=106

21번째 기사글부터 크롤링을 시작합니다.
Processing article : 어제의 적이 오늘의 동지…"시장 가세요" 대형마트 팔 걷은 이유
Processing article : 한국 남성, ‘이 나이대’ 근육량 뚝↓…노화 시작된다
*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : https://n.news.naver.com/mnews/article/312/0000602974?sid=106
*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : https://n.news.naver.com/mnews/article/057/0001731053?sid=106

31번째 기사글부터 크롤링을 시작합니다.
Processing article : 정부, 내년도 예산안 편성 절차 돌입…작성 지침 발표
Processing article : MS, 블리자드 인수 ‘한시름’…英당국, 긍정 입장 선회
*** 다음 링크의 뉴스를 크롤링

,Title,Date,Article,URL,PressCompany
0,"AI가 핫하다는데 어디에 투자를, 110조로 커지는 이 시장 [더테크웨이브]",2023.03.25. 오후 6:05,클래리베이트 ‘글로벌 AI반도체 혁신 보고서’ AI 반도체 [사진=연합뉴스]최근 챗...,https://n.news.naver.com/mnews/article/009/000...,매일경제
1,[건강당당] 숫자로 알아보는 당뇨,2023.03.25. 오후 5:01,"[그래픽=최소연 디자이너]고려대 의대 윤석준 교수팀은 국민건강보험, 중앙암등록본부,...",https://n.news.naver.com/mnews/article/296/000...,코메디닷컴
2,"VR로 재탄생한 1978년 게임, 집처럼 편안한 혼합현실..""VR은 살아있다""",2023.03.25. 오후 4:56,[GDC23]게임개발자 콘퍼런스서 VR 기업 존재감고전게임 '시조새' 아타리 VR게...,https://n.news.naver.com/mnews/article/018/000...,이데일리
3,"‘미국 퇴출 위기’ 틱톡, 정작 중국인은 ‘이것’ 때문에 안 쓴다 [핫이슈]",2023.03.25. 오후 4:28,"[서울신문 나우뉴스]왼쪽 더우인, 오른쪽 틱톡 미국 정치권에서 중국산 동영상 플랫폼...",https://n.news.naver.com/mnews/article/081/000...,서울신문
4,"美 퇴출위기 맞은 틱톡… ""바이든 최측근 고용했다""",2023.03.25. 오후 3:36,미국 시장에서 퇴출 압박을 받고 있는 틱톡이 조 바이든 대통령의 측근들을 잇따라 고...,https://n.news.naver.com/mnews/article/417/000...,머니S


In [45]:
# "데이터 분석"이라는 6글자의 문구가 반드시 포함된 문서만 검색하려면?

query = '?데이터 분석?'

<br>
<br>

## (Appendix) 깔끔하게 정리하기

In [26]:
def main_crawling(query, start_date, end_date, sort_type, max_page):
    
    if query == '':
        query = '데이터 분석'
    if len(start_date) != 10:
        start_date = '2021.01.01'
    if len(end_date) != 10:
        end_date = '2021.12.31'
    if sort_type not in ['0', '1', '2']:
        sort_type = '0'
#     if max_page == '':
#         max_page = 5


    # 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
    titles = []
    dates = []
    articles = []
    article_urls = []
    press_companies = []

    # 주어진 일자를 쿼리에 맞는 형태로 변경해줍니다.
    start_date = start_date.replace(".", "")
    end_date = end_date.replace(".", "")

    # 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
    current_call = 1
    last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당 


    while current_call <= last_call:

        print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))

        url = "https://search.naver.com/search.naver?where=news&query=" + query \
              + "&sort=" + sort_type \
              + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
              + "to" + end_date \
              + "%2Ca%3A&start=" + str(current_call)

        urls_list = []
        try: # 네이버 뉴스 검색결과 페이지 자체에 접근이 불가능할 경우 에러가 발생할 수 있습니다.
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')

            for urls in source.find_all('a', {'class' : "info"}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    urls_list.append(urls["href"])
        except:
            print('해당 뉴스 검색 페이지의 네이버 뉴스 링크를 모으는 중 에러가 발생했습니다. : ', url)
        
        # urls_list : 해당 페이지에 있는 "네이버 뉴스"의 링크 모음(list)
        if urls_list != []:
            for url in urls_list:
                try: # 특정 뉴스 기사글 하나를 크롤링하는 중 에러가 발생할 수 있습니다.ㄴ
                    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
                    web_news = requests.get(url, headers=headers).content
                    source_news = BeautifulSoup(web_news, 'html.parser')

                    title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
                    print('Processing article : {}'.format(title))

                    date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

                    article = source_news.find('div', {'id' : 'dic_area'}).get_text()
                    article = article.replace("\n", "")
                    article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                    article = article.replace("동영상 뉴스       ", "")
                    article = article.replace("동영상 뉴스", "")
                    article = article.strip()

                    press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()

                    titles.append(title)
                    dates.append(date)
                    articles.append(article)
                    press_companies.append(press_company)
                    article_urls.append(url)
                except:
                    print('\n*** {}번부터 {}번까지의 기사글을 크롤링하는 중 문제가 발생했습니다.'.format(current_call, current_call+9))
                    print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
        else:
            pass

        time.sleep(5)
        current_call += 10
            
            
    article_df = pd.DataFrame({'Title':titles, 
                               'Date':dates, 
                               'Article':articles, 
                               'URL':article_urls, 
                               'PressCompany':press_companies})
    now = datetime.now()
    kor_now = datetime(now.year, now.month, now.day, 
    now.hour + 9 if now.hour + 9 < 24 else now.hour + 9 -24 , 
    now.minute, now.second)
    article_df.to_excel('result_{}.xlsx'.format(kor_now.strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
    
    print('\n크롤링이 성공적으로 완료되었습니다!')
    print('\n크롤링 결과를 다음 파일에 저장하였습니다 : {}'.format(kor_now.strftime('%y%m%d_%H%M')))

In [23]:

'''A.year = 
A.month = 
A.hour = datetime.now().hour + 8
A.minute = datetime.now().minute
A.second = datetime.now().second'''
print(datetime.now())
print(A)

2023-03-26 00:43:07.039230
2023-03-26 09:43:07


In [0]:
input()

In [0]:
input('나이를 적어주세요.')

In [0]:
age = input("나이를 적어주세요.")
print('나이 x 2는 ', age * 2)

In [0]:
age = input("나이를 적어주세요.")
type(age)

In [0]:
age = int(input("나이를 적어주세요."))
print('나이 x 2는 ', age * 2)

In [0]:
age = input() # 아무 내용도 입력하지 않고 enter 시 NoneType이 아닌 ''(str)가 리턴됩니다.
print(type(age))
age # age == ""

In [27]:
query = input('검색어를 입력해주세요. (ex. 데이터 분석) : ')
start_date = input('검색 시작 날짜를 입력해주세요. (형식 : 2021.01.01) : ')
end_date = input('검색 종료 날짜를 입력해주세요. (형식 : 2021.12.31) : ')
sort_type = input('정렬 타입을 입력해주세요 (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : ')
max_page = input('크롤링을 원하는 전체 페이지 수를 입력해주세요. (ex. 5) : ')

if start_date > end_date:
    print('\n시작 날짜는 종료 날짜보다 이후로 지정하실 수 없습니다. 다시 실행해주세요!')
elif max_page == '':
    max_page = 5
    print('\n원하시는 페이지 수가 입력되지 않았습니다. 5 페이지까지만 크롤링을 진행합니다.')
    main_crawling(query, start_date, end_date, sort_type, max_page)
else:
    max_page = int(max_page)
    main_crawling(query, start_date, end_date, sort_type, max_page)


1번째 기사글부터 크롤링을 시작합니다.
Processing article : 구글 인공지능 바드, 챗GPT보다 훨씬 신중-NYT분석
Processing article : 3월부터 시작한 택시요금 인상 충격...승객 33% 급락 [데이터로 본 대한민국]
Processing article : [게시판] 네이버, 쇼핑데이터로 건강기능식품 추천 기획전
Processing article : 韓 민간 발사체 '한빛-TLV', 정상 이륙…비행 데이터 분석 중
Processing article : 올해 뉴스 분석하는 챗GPT가 있다고?…챗GPT 플러그인 출시
Processing article : KTNET-산업연구원, 데이터 기반 사업 협력
Processing article : 3D프린팅 기술 비롯, 드론으로 수중위험 예방·하늘서 데이터 수집, …진화하는 스마트건설

11번째 기사글부터 크롤링을 시작합니다.
Processing article : 인바디, 전 세계 체성분 분석 빅데이터 보고서 발간
Processing article : 광주 테크노파크, 치매 데이터 실증 기반 구축 속도
Processing article : "데이터시장 규모 2027년 50조…경남 통합데이터 구축 필요"
Processing article : 한전, 전력 데이터 활용 활성화 위한 설명회 개최
Processing article : 관광공사 "데이터랩 신규 데이터 수집 확대…분석 기능 강화"
Processing article : 메디에이지-스위스리 한국지점, 의료 빅데이터 기반 공동연구 협약 체결
Processing article : 코인원, 임직원 데이터 분석 능력 증진 사내 교육 진행

크롤링이 성공적으로 완료되었습니다!

크롤링 결과를 다음 파일에 저장하였습니다 : 230326_0948


In [0]:
df = pd.read_excel('?.xlsx') 
df.head()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b844bbc7-67c4-4320-a052-bd9703f8a211' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>